# Model Training

In this notebook, we will ask you a series of questions regarding model selection. Based on your responses, we will ask you to create the ML models that you've chosen. 

The bonus step is completely optional, but if you provide a sufficient third machine learning model in this project, we will add `1000` points to your Kahoot leaderboard score.

**Note**: Use the dataset that you've created in your previous data transformation step (not the original model).

## AI/ML Model Training Steps/workflow:
1. Import the necessary libraries
2. Load data
3. Perform EDA
4. Preprocess data
5. Train the model
   - train/test/split
   - Search
   - Make prediction , and then 
6. Evaluate the model

- ☑️ Load data
- ☑️ Split train/test (keep fraud ratio with stratify)
- ☑️ Balance minority fraud class with SMOTE
- ☑️ Scale features for all models
- ☑️ Define classifiers in a dictionary
- ☑️ Loop: search for best parameters using RandomizedSearchCV
- ☑️ Loop: train, predict, evaluate
- ☑️ Print Confusion Matrix & Classification Report for each

In [ ]:
# Import necessary libraries and modules
# Panda for handliing dataframes and numpy for handling arrays
import pandas as pd
import numpy as np

# Load models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Preprocessing SMOTE Oversampling and Scaling module 
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Max num n_jobs/processor(s) that can be used
from joblib import cpu_count
print(f"There is currently {cpu_count()} CPU-Cores in Use for training")

# Load evaluation metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
 
# Import modules for data manipulation
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

There is currently 8 CPU-Cores in Use for training


In [24]:
# read in csv file to pandas dataframe
df_baseline_hot_encode = pd.read_csv("../data/baseline_hot_encode.csv")

In [25]:
# first five rows
df_baseline_hot_encode.head()

,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,40490.46,100928.00,141418.46,242868.95,272554.26,0,False,False,False,False
1,350188.68,5056.00,0.00,4707.66,354896.34,0,True,False,False,False
2,2224.85,2578.65,353.80,0.00,0.00,0,False,False,True,False
3,170020.34,2096782.65,2266802.99,2668162.89,2861708.82,0,False,False,False,False
4,261231.80,2215631.50,2476863.31,493557.37,232325.57,0,False,False,False,False


## Questions
Is this a classification or regression task?  

This is a classification task since we are effectively assigning one of two cases to either respective 'bin' of fraud and not-fraud (i.e., 0 or 1).

Are you predicting for multiple classes or binary classes?  

Since this is a binary classification problem, where we are trying to find out if transactions are either fraud or not-fraud, there are only 2 possible classes/labelings.

Given these observations, which 2 (or possibly 3) machine learning models will you choose?  

I'm using Logistic Regression, Support Vector Machine (SVM), and AdaBoost.

## First Model

Using the first model that you've chosen, implement the following steps.

### 1) Create a train-test split

Use your cleaned and transformed dataset to divide your features and labels into training and testing sets. Make sure you’re only using numeric or properly encoded features.  

In [ ]:
# Imbalance Classification Problem:
# numerical columns for use as predictors: amount oldbalanceOrg newbalaceOrig oldbalalceDest newbalanceDest

# Step 2: Separate features (X) and target (y)
X = df_baseline.drop(columns=['isFraud'])  # All features except target
y = df_baseline['isFraud']                 # Target variable

# Step 3: Split into training and testing sets
# Use stratify=y to keep the fraud/non-fraud ratio consistent, but I think could be reduntdant when using SMOTE??
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 4: Balance the training set using SMOTE
# SMOTE generates synthetic minority class (fraud) samples
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# ✅ Step 5: Scale the features (mean=0, variance=1)
# Fit scaler only on training data to prevent data leakage
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)  # Apply same scaler to test set

print("✅ Data pre-processing complete: SMOTE applied and features scaled.")
print("📊 Class balance after SMOTE:\n", y_resampled.value_counts())  

✅ Data pre-processing complete: SMOTE applied and features scaled.
📊 Class balance after SMOTE:
 isFraud
0    159793
1    159793
Name: count, dtype: int64


### 2) Search for best hyperparameters
Use tools like GridSearchCV, RandomizedSearchCV, or model-specific tuning functions to find the best hyperparameters for your first model.

In [ ]:
# Step 6: hyperparameter Search Grids
param_grids = {
    "Random Forest": {
        "n_estimators": [100, 200, 500],
        "criterion": ["gini", "entropy", "log_loss"],
        "max_depth": range(10, 50, 5),
        "max_features": ["sqrt", "log2"],
        "min_samples_split": [2, 5, 10]
    },
    "AdaBoost": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 1, 2]
    },
    "Logistic Regression": {
        "C": np.logspace(-3, 3, 7),
        "penalty": ["l1", "l2"],
        "solver": ["liblinear"]
    }
}



### 3) Train your model
Select the model with best hyperparameters and generate predictions on your test set. Evaluate your models accuracy, precision, recall, and sensitivity.  

In [ ]:
# Step 6: Define classifiers
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42)
}

# Store best models for future use and less compute
best_models = {}

# Select models for training and tuning
for name, model in models.items():
    print(f"\n🔥 Training and tuning: {name}")
    
    param_grid = param_grids[name] # Select which param to use in randomized search
    
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=10,  # Tuning param
        cv=5,
        scoring='f1',  # Scoring param, maybe I can also try 'roc_auc'??
        random_state=42,
        n_jobs=-1 # Number of CPU-Cores param 
    )
    
    # Fit to training data
    random_search.fit(X_resampled_scaled, y_resampled)
    
    # Save the best model
    best_models[name] = random_search.best_estimator_

    # Predict on test set using the best model
    yhat = best_models[name].predict(X_test_scaled)
    
    # Evaluate
    cm = confusion_matrix(y_test, yhat)
    report = classification_report(y_test, yhat, digits=3)
    
    print(f"✅ Best Parameters for {name}:\n", random_search.best_params_)
    print("📊 Confusion Matrix:\n", cm)
    print("\n📋 Classification Report:\n", report)



💥 Training and evaluating: Random Forest
📊 Confusion Matrix:
 [[39820   128]
 [   11    41]]

📄 Classification Report:
               precision    recall  f1-score   support

           0      1.000     0.997     0.998     39948
           1      0.243     0.788     0.371        52

    accuracy                          0.997     40000
   macro avg      0.621     0.893     0.685     40000
weighted avg      0.999     0.997     0.997     40000


💥 Training and evaluating: AdaBoost
📊 Confusion Matrix:
 [[37111  2837]
 [    1    51]]

📄 Classification Report:
               precision    recall  f1-score   support

           0      1.000     0.929     0.963     39948
           1      0.018     0.981     0.035        52

    accuracy                          0.929     40000
   macro avg      0.509     0.955     0.499     40000
weighted avg      0.999     0.929     0.962     40000


💥 Training and evaluating: Logistic Regression
📊 Confusion Matrix:
 [[39049   899]
 [    5    47]]

📄 Classi

## Second Model

Create a second machine learning object and rerun steps (2) & (3) on this model. Compare accuracy metrics between these two models. Which handles the class imbalance more effectively?

Create as many code-blocks as needed.

In [ ]:
# Three models are in the above cell

### (Bonus/Optional) Third Model

Create a third machine learning model and rerun steps (2) & (3) on this model. Which model has the best predictive capabilities? 

Create as many code-blocks as needed.

In [15]:
# initialize a Support Vector Classifier with RBF kernel to handle non-linearity
svc_non_linear = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# Train the classifier on the XOR dataset
svc_non_linear.fit(X_train, y_train)

# make predictions on the same dataset
yhat = svc_non_linear.predict(X_test)

confusion = confusion_matrix(y_test, yhat)
class_report = classification_report(y_test, yhat)

print("Confusion Matrix \n", confusion)
print("\nClassification Report:\n", class_report)

Confusion Matrix 
 [[39948     0]
 [   42    10]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     39948
           1       1.00      0.19      0.32        52

    accuracy                           1.00     40000
   macro avg       1.00      0.60      0.66     40000
weighted avg       1.00      1.00      1.00     40000



In [ ]:
# implement random search on the LinearSVC model to find best hyperparams
svc_params_grid = {
    'C': np.linspace(0.1, 10, 100),
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4, 5]  
}

svc = SVC(max_iter=10000, random_state=42)

# set up RandomizedSearchCV with 5-fold cross-validation
random_search = RandomizedSearchCV(svc, param_distributions=svc_params_grid, scoring='f1', n_jobs=-1, cv=5, random_state=42)

# fit this model on your training data
random_search.fit(X_train, y_train)  # ran for 36 mins without n_jobs param being set...

c:\Users\oneps\anaconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\oneps\anaconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\oneps\anaconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\oneps\anaconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\oneps\anaconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:305: Convergence

RandomizedSearchCV(cv=5, estimator=SVC(max_iter=10000, random_state=42),
                   param_distributions={'C': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,
        6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,
        7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,
        8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9,
       10. ]),
                                        'degree': [2, 3, 4, 5],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['rbf', 'poly', 'sigmoid']},
                   random_state=42)

In [17]:
best_svc = random_search.best_estimator_

# make predictions on the same dataset
yhat = best_svc.predict(X_test)

confusion = confusion_matrix(y_test, yhat)
class_report = classification_report(y_test, yhat)

print("Confusion Matrix \n", confusion)
print("\nClassification Report:\n", class_report)

Confusion Matrix 
 [[39947     1]
 [   36    16]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     39948
           1       0.94      0.31      0.46        52

    accuracy                           1.00     40000
   macro avg       0.97      0.65      0.73     40000
weighted avg       1.00      1.00      1.00     40000

